In [359]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler, MinMaxScaler, binarize, normalize
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import accuracy_score
from collections import Counter
import time
import sys

In [334]:
df = pd.read_csv('./sample_data/data.csv',header=None)
data=df.values
y_true = pd.read_csv('./sample_data/label.csv',header=None).values
K=10

In [335]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

In [336]:
def cosine_distance(x1, x2):
    return 1 - cosine_similarity(x1.reshape(1, -1), x2.reshape(1, -1))[0, 0]

In [337]:
def jaccard_distance(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    return 1 - (intersection / union)

In [338]:
def kmeans(X, k, distance_measure='euclidean', max_iters=100):
    n_samples, n_features = X.shape

    # Initialize centroids
    centroids = X[np.random.choice(n_samples, k, replace=False)]
    prev_centroids = centroids.copy()

    # Stop criteria variables
    max_iters_reached = False
    sse_increase = False
    no_change_in_centroids = False

    # Track iterations and time
    iterations = 0
    start_time = time.time()

    for _ in range(max_iters):
        # Assign each sample to the nearest centroid
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_measure == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'cosine':
                distances[:, i] = np.array([cosine_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'jaccard':
                distances[:, i] = np.array([jaccard_distance(x, centroids[i]) for x in X])

        labels = np.argmin(distances, axis=1)

        # Update centroids
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])

        # Check for convergence
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids
        # Increment iterations
        iterations += 1

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Calculate Sum of Squared Errors (SSE)
    if distance_measure == 'euclidean':
        sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
    elif distance_measure == 'cosine':
        sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
    elif distance_measure == 'jaccard':
        sse = np.sum(np.sum([jaccard_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))

    return labels, centroids, iterations,elapsed_time,sse


In [339]:
# Euclidean distance
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data)
labels_euclidean, centroids_euclidean, iters_euclidean, time_euclidean,sse_euclidean = kmeans(data_standardized, K, 'euclidean')

<ipython-input-338-12cd3a025a4d>:46: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [340]:
print(sse_euclidean)

5572509.488383138


In [341]:
data_normalized = normalize(data)
labels_cosine, centroids_cosine, iters_cosine, time_cosine,sse_cosine = kmeans(data_normalized, K, 'cosine')

<ipython-input-338-12cd3a025a4d>:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [342]:
print(sse_cosine)

688.0471424786685


In [343]:
data_binarized = binarize(data)
labels_jaccard, centroids_jaccard, iters_jaccard, time_jaccard,sse_jaccard  = kmeans(data_binarized, K, 'jaccard')

<ipython-input-338-12cd3a025a4d>:50: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([jaccard_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [344]:
print(f"Euclidean K-means SSE: {sse_euclidean}")
print(f"Cosine K-means SSE: {sse_cosine}")
print(f"Jaccard K-means SSE: {sse_jaccard}")

Euclidean K-means SSE: 5572509.488383138
Cosine K-means SSE: 688.0471424786685
Jaccard K-means SSE: 3019.7956677161465


In [345]:
def majority_vote_labels(cluster_labels):
    counter = Counter(map(tuple, cluster_labels))
    majority_label = counter.most_common(1) [0] [0]
    return majority_label

In [347]:
def compute_accuracy(predicted_labels, true_labels):
    majority_vote_dict = {}
    for cluster_label in np.unique(predicted_labels):
        cluster_indices = np.where(predicted_labels == cluster_label)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_vote_dict[cluster_label] = majority_vote_labels(cluster_true_labels)

    predicted_majority_labels = np.array([majority_vote_dict[label] for label in predicted_labels])
    accuracy = np.mean(predicted_majority_labels == true_labels)
    return accuracy

In [348]:
accuracy_euclidean = compute_accuracy(labels_euclidean, y_true)
accuracy_cosine = compute_accuracy(labels_cosine, y_true)
accuracy_jaccard = compute_accuracy(labels_jaccard, y_true)

In [349]:
# Compare accuracies
print("Euclidean Accuracy:", accuracy_euclidean)
print("Cosine Accuracy:", accuracy_cosine)
print("Jaccard Accuracy:", accuracy_jaccard)

Euclidean Accuracy: 0.5162
Cosine Accuracy: 0.5496
Jaccard Accuracy: 0.6185


In [350]:
# Compare iterations and time
print("Euclidean-K-means: Iterations =", iters_euclidean, ", Time =", time_euclidean)
print("Cosine-K-means: Iterations =", iters_cosine, ", Time =", time_cosine)
print("Jaccard-K-means: Iterations =", iters_jaccard, ", Time =", time_jaccard)

Euclidean-K-means: Iterations = 39 , Time = 34.43560814857483
Cosine-K-means: Iterations = 88 , Time = 1962.5294678211212
Jaccard-K-means: Iterations = 60 , Time = 84.18351721763611


In [351]:
def kmeans_iterations(X, k, distance_measure='euclidean', max_iters=100):
    n_samples, n_features = X.shape

    # Initialize centroids
    centroids = X[np.random.choice(n_samples, k, replace=False)]
    prev_centroids = centroids.copy()

    # Stop criteria variables
    max_iters_reached = False
    sse_increase = False
    no_change_in_centroids = False

    # Track iterations and time
    iterations = 0
    start_time = time.time()

    for _ in range(max_iters):
        # Assign each sample to the nearest centroid
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_measure == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'cosine':
                distances[:, i] = np.array([cosine_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'jaccard':
                distances[:, i] = np.array([jaccard_distance(x, centroids[i]) for x in X])

        labels = np.argmin(distances, axis=1)

        # Update centroids
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])

        # Check for convergence
        if iterations == max_iters-1:
            break

        centroids = new_centroids
        # Increment iterations
        iterations += 1

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Calculate Sum of Squared Errors (SSE)
    if distance_measure == 'euclidean':
        sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
    elif distance_measure == 'cosine':
        sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
    elif distance_measure == 'jaccard':
        sse = np.sum(np.sum([jaccard_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))

    return labels, centroids, iterations,elapsed_time,sse


In [354]:
# Euclidean distance
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data)
labels_euclidean, centroids_euclidean, iters_euclidean, time_euclidean,sse_euclidean = kmeans_iterations(data_standardized, K, 'euclidean',max_iters=10)

<ipython-input-351-cc99128566a1>:46: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [355]:
data_normalized = normalize(data)
labels_cosine, centroids_cosine, iters_cosine, time_cosine,sse_cosine = kmeans_iterations(data_normalized, K, 'cosine',max_iters=10)

<ipython-input-351-cc99128566a1>:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [357]:
data_binarized = binarize(data)
labels_jaccard, centroids_jaccard, iters_jaccard, time_jaccard,sse_jaccard  = kmeans_iterations(data_binarized, K, 'jaccard',max_iters=10)

<ipython-input-351-cc99128566a1>:50: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sse = np.sum(np.sum([jaccard_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [358]:
print(f"Euclidean K-means SSE after 10 iterations: {sse_euclidean}")
print(f"Cosine K-means SSE after 10 iterations: {sse_cosine}")
print(f"Jaccard K-means SSE after 10 iterations: {sse_jaccard}")

Euclidean K-means SSE after 10 iterations: 5586842.9976208005
Cosine K-means SSE after 10 iterations: 700.2502635282805
Jaccard K-means SSE after 10 iterations: 3032.9003283724637


In [360]:
def kmeans_iterations(X, k, distance_measure='euclidean', max_iters=100):
    n_samples, n_features = X.shape

    # Initialize centroids
    centroids = X[np.random.choice(n_samples, k, replace=False)]
    prev_centroids = centroids.copy()

    # Stop criteria variables
    max_iters_reached = False
    sse_increase = False
    no_change_in_centroids = False

    # Track iterations and time
    iterations = 0
    start_time = time.time()
    sse = sys.float_info.max
    new_sse = 0

    for _ in range(max_iters):
        # Assign each sample to the nearest centroid
        distances = np.zeros((n_samples, k))
        for i in range(k):
            if distance_measure == 'euclidean':
                distances[:, i] = np.array([euclidean_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'cosine':
                distances[:, i] = np.array([cosine_distance(x, centroids[i]) for x in X])
            elif distance_measure == 'jaccard':
                distances[:, i] = np.array([jaccard_distance(x, centroids[i]) for x in X])

        labels = np.argmin(distances, axis=1)

        # Update centroids
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])
        # Calculate Sum of Squared Errors (SSE)
        if distance_measure == 'euclidean':
            new_sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
        elif distance_measure == 'cosine':
            new_sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
        elif distance_measure == 'jaccard':
            new_sse = np.sum(np.sum([jaccard_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
        # Check for convergence
        if new_sse>sse:
            break

        sse = new_sse
        # Increment iterations

    end_time = time.time()
    elapsed_time = end_time - start_time



    return labels, centroids, iterations,elapsed_time,sse


In [363]:
# Euclidean distance
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data)
labels_euclidean, centroids_euclidean, iters_euclidean, time_euclidean,sse_euclidean = kmeans_iterations(data_standardized, K, 'euclidean',max_iters=500)

<ipython-input-360-eacaed9768d6>:36: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  new_sse = np.sum(np.sum([euclidean_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))


In [ ]:
data_normalized = normalize(data)
labels_cosine, centroids_cosine, iters_cosine, time_cosine,sse_cosine = kmeans_iterations(data_normalized, K, 'cosine',max_iters=500)

<ipython-input-360-eacaed9768d6>:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  new_sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
<ipython-input-360-eacaed9768d6>:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  new_sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(len(centroids)))
<ipython-input-360-eacaed9768d6>:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  new_sse = np.sum(np.sum([cosine_distance(x, centroids[i]) ** 2 for x in X[labels == i]]) for i in range(

In [ ]:
data_binarized = binarize(data)
labels_jaccard, centroids_jaccard, iters_jaccard, time_jaccard,sse_jaccard  = kmeans_iterations(data_binarized, K, 'jaccard',max_iters=500)

In [ ]:
print(f"Euclidean K-means SSE after SSE increase: {sse_euclidean}")
print(f"Cosine K-means SSE after SSE increase: {sse_cosine}")
print(f"Jaccard K-means SSE after SSE increase: {sse_jaccard}")